<a href="https://colab.research.google.com/github/AishwaryaKaranth/final-year-project/blob/master/final_year_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


##Resizing

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:
!unzip '/content/gdrive/MyDrive/Final year Project/Galaxy Classification/Dataset/images_training_rev1.zip' -d '/content/gdrive/MyDrive/Final year Project/Galaxy Classification/Dataset/Training set/'

In [ ]:
!pip install Pillow

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import PIL
from PIL import Image
import os

In [ ]:
def Resizing(path1,currentpath):
  for im in os.listdir(path1):
    try:
      img=Image.open(os.path.join(path1,im))
      resized=img.resize((224,224))
      resized.save(currentpath+'Resized/'+im[:-4]+'resized.jpg', 'JPEG',quality=90)
      print('resizing')
    except Exception as e:
      print(str(e))

In [ ]:
currentpath='/content/gdrive/MyDrive/Final year Project/Galaxy Classification/Dataset/'
path1=currentpath+'Training set/images_training_rev1/'

In [ ]:
Resizing(path1,currentpath)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import os
tr=os.listdir(path1)
re=os.listdir(currentpath+'Resized/Resized_Training/')
print(len(tr),' , ',len(re))

61578  ,  61578


All the 61578 images in the training set have been resized to 224*224


##Data Augmentation by Rotation

In [ ]:
currentpath='/content/gdrive/MyDrive/Final year Project/Galaxy Classification/Dataset/'
path1=currentpath+'Resized/Resized_Training/'

In [ ]:
import PIL
from PIL import Image
import os
import random

In [ ]:
def DA(path1,currentpath):
  for im1 in os.listdir(path1):
    img=Image.open(os.path.join(path1,im1))
    for i in range(1,5):
      try:
        angle=random.uniform(1,360)
        rot = img.rotate(angle)
        rot.save(currentpath+'/Rotation/'+im1[:-4]+str(i)+'rotated.jpg', 'JPEG',quality=90)
        print('rotating')
      except Exception as e:
        print(str(e))

In [ ]:
DA(path1,currentpath)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import os
path2='/content/gdrive/MyDrive/Galaxy Classification Code/Rotation/'
rot=os.listdir(path2)
print(61578*4,' , ',len(rot))

There are 61578*4=246312 images now in the dataset


# Median Filtering

In [ ]:
import PIL
from PIL import Image,ImageFilter
import os

In [ ]:
def median_cal(path1,path2):
    images=os.listdir(path1)
    for image in images:
        full_path=os.path.join(path1,image)
        try :
            print(full_path)
            I=Image.open(full_path)
            modified= I.filter(ImageFilter.MedianFilter(size = 3))  
            destn=os.path.join(path2,str(image[:-4]))
            destn+='_md.jpg'
            print(destn)
            modified.save(destn,"JPEG",quality=90)
        except Exception as e:
            print(str(e))


In [ ]:
path1='./Rotation'
len(os.listdir(path1))
path2='./Median'
median_cal(path1,path2)


#Contrast Limited Adaptive Histogram Equalization(CLAHE)

In [ ]:
import os    
import cv2
import numpy as np

In [ ]:
def he_Equ(path1,path2):
    images=os.listdir(path1)
    for image in images:
        full_path=os.path.join(path1,image)
        try :
            orig=cv2.imread(full_path)               # Loading Source File images.
            ycb=cv2.cvtColor(orig,cv2.COLOR_BGR2LAB) # Converting RGB to LAB colorscale
            imgs=cv2.split(ycb)                      # Splitting Channels of LAB Colorscale. 
            clahe=cv2.createCLAHE(clipLimit=2.0,tileGridSize=(3,3))  # Creating CLAHE(Clip limit:sets threshhold for contrast limiting.tilegrid size:used for subimages.)
            imgs[0]= clahe.apply(imgs[0])            # Applying Clahe on luminescence channel
            lab=cv2.merge(imgs)                      # Merging channels of LAB.
            lab=cv2.cvtColor(lab,cv2.COLOR_LAB2BGR)  # Converting back LAB to RGB color scale.
            destn=os.path.join(path2,str(image[:-4]))
            destn=destn+'_he.jpg'              
            print(destn)
            cv2.imwrite(destn,lab)                   # Storing the HE images in Destination Folder.
        except Exception as e:
            print(str(e)) 

In [ ]:
path1='./MedianFilter'
len(os.listdir(path1))
path2='./he1'
he_Equ(path1,path2)

# VGG16 Architecture

In [ ]:
import keras
from keras.models import Sequential, Model
from keras.layers.core import Flatten, Dense, Dropout, Lambda, Reshape
from keras.layers import Input
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Conv2D, MaxPooling2D, Activation
from keras.optimizers import RMSprop

In [ ]:
def ConvLayers(layers, model, filters):
  for i in range(layers):
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(filters, 3,3,activation='relu'))
  model.add(MaxPooling2D((2,2),strides=(1,1),padding="same"))

In [ ]:
def FCLayers(model):
  model.add(Dense(4096, activation='relu'))
  model.add(Dropout(0.5))

In [ ]:
def VGG16_arc():
  model=Sequential()
  ConvLayers(2,model,64)
  ConvLayers(2,model,128)
  ConvLayers(3,model,256)
  ConvLayers(3,model,512)
  ConvLayers(3,model,512)
  model.add(Flatten())
  FCLayers(model)
  FCLayers(model)
  model.add(Dense(37,activation='sigmoid'))
  #model.add(Dense(3,activation='softmax'))
  return model

In [ ]:
#Model compilation
optimizer = RMSprop(lr=1e-6)#can be later tweaked
model = VGG16_arc()
model.compile(loss='mean_squared_error', optimizer=optimizer) 
#model.compile(loss='categorical_crossentropy', optimizer=optimizer)
